In [1]:
from __future__ import division
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import random
from random import randint
import nltk
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import time
import gensim
from gensim.models import FastText, Word2Vec
import io
import collections
import math
import argparse
import string

/home/cules/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

In [3]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
def tokenize(s):
    return tokens_re.findall(s)

In [37]:
from GetData import getData
data_rows,sents,addict_label=getData()

In [38]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sents)
sequences = tokenizer.texts_to_sequences(sents)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 12772 unique tokens.


In [39]:
X=pad_sequences(sequences, padding='pre')

In [40]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(addict_label)

In [41]:
from keras.utils import to_categorical
labels = np.asarray(labels)
#SHUFFLING
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
labels = labels[indices]
MAX_SEQUENCE_LENGTH = X.shape[1]

In [42]:
le.inverse_transform([0,1,2,3,4])

/home/cules/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['Addicted', 'Addiction-prone', 'NA', 'Recovered', 'Recovering'],
      dtype='<U15')

In [43]:
# embeddings_index = {}
ft_embed = FastText.load('./../models/first_eval_dataset_fasttext.model')
w2v_embed = Word2Vec.load('./../models/first_eval_dataset_word2vec.model')
# f = open('glove.twitter.27B.100d.txt','r')
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

In [44]:
EMBEDDING_DIM=150
not_found=[]
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for (word, i) in word_index.items():
    embedding_vector = None
    try:
        embedding_vector = ft_embed[word]
    except:
        embbedding_vector = None
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        not_found.append(word)

/home/cules/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [45]:
len(X)

727

In [46]:
embedding_matrix[0] # The paddings in the sequence correspond to this

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [47]:
not_found # words not found

['i',
 'a',
 'my',
 'if',
 'i’m',
 't',
 'i’d',
 'd',
 'm',
 '“i',
 '3d',
 '3g',
 '“my',
 '“no',
 '29',
 'btw',
 '1g',
 'uk',
 '4mmc',
 '“i’m',
 '5htp',
 'n',
 '2g',
 '99',
 '67',
 'nsfw',
 'mr',
 '44',
 '39',
 'nyc',
 '65',
 '“do',
 'μg',
 'pbj',
 '48',
 '72',
 'eh',
 'uv',
 'iq',
 'hbwr',
 '46',
 '31',
 '92',
 'fbi',
 '79',
 'o',
 'gf',
 'y',
 's',
 'uh',
 '66',
 '47',
 'ua',
 '77',
 '“a',
 '5h',
 'ii',
 '52',
 '38',
 '4d',
 'xr',
 '8a',
 'w',
 'mcg',
 '📷',
 '♥',
 'µg',
 'dj',
 'i՚m',
 'fs',
 'uhh',
 'fb',
 'gta5',
 '5x3',
 'km',
 '2k',
 'dvd',
 '“yeh”',
 'lmfao',
 '·',
 'cpr',
 'ipa',
 'bmw',
 'i5',
 'v',
 '😏',
 '🙌🏼',
 '1k',
 '43',
 'ssdi',
 '💕',
 '🎶',
 '🔥🔥',
 '🎉',
 'k2',
 'xo',
 'iii',
 '08',
 'gpa',
 '7991',
 '‘my',
 '9k',
 '7686',
 'xyz',
 '3s',
 '7s',
 'zu',
 '6apdb',
 '97',
 '49',
 'cdc',
 'tp',
 'q4',
 'mmj',
 'l',
 '😉',
 'nba',
 '78',
 '09',
 '6’2”',
 'bp',
 't10h',
 '6abp',
 '8b',
 't7',
 'ltr',
 't3',
 't4',
 '57',
 '5x',
 '56',
 '94',
 't0055',
 '9lb',
 'gvg',
 'uo',
 '943

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33, random_state=42)

seed = 21
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
f_cvscores = []
prec_cvscores = []
recall_cvscores = []
iteration_count=1
print("Iteration count",iteration_count)
iteration_count+=1
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,trainable=False))
model.add(LSTM(180,dropout=0.25, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_labels_train = to_categorical(np.asarray(y_train))
model_labels_test = to_categorical(np.asarray(y_test))
model.fit(X_train, model_labels_train, epochs=9)
y_pred=model.predict(X_test)
pred = np.zeros(y_pred.shape)
for i in range(y_pred.shape[0]):
    ind = np.argmax(y_pred[i])
    pred[i][ind] = 1


f_cvscores.append(precision_recall_fscore_support(model_labels_test,pred,average='weighted')[2])
prec_cvscores.append(precision_recall_fscore_support(model_labels_test,pred,average='weighted')[0])
recall_cvscores.append(precision_recall_fscore_support(model_labels_test,pred,average='weighted')[1])
    

Iteration count 1
Epoch 1/9
487/487 [==============================] - 107s 219ms/step - loss: 0.4739 - acc: 0.7984
Epoch 2/9
487/487 [==============================] - 71s 146ms/step - loss: 0.4541 - acc: 0.7979
Epoch 3/9
487/487 [==============================] - 80s 165ms/step - loss: 0.4408 - acc: 0.8041
Epoch 4/9
487/487 [==============================] - 77s 159ms/step - loss: 0.4236 - acc: 0.8094
Epoch 5/9
487/487 [==============================] - 81s 167ms/step - loss: 0.4101 - acc: 0.8197
Epoch 6/9
487/487 [==============================] - 104s 214ms/step - loss: 0.4029 - acc: 0.8230
Epoch 7/9
487/487 [==============================] - 74s 152ms/step - loss: 0.3945 - acc: 0.8251
Epoch 8/9
448/487 [==========================>...] - ETA: 6s - loss: 0.3912 - acc: 0.8272 

In [ ]:
np.mean(prec_cvscores)

In [ ]:
np.mean(recall_cvscores)

In [ ]:
np.mean(f_cvscores)

In [109]:
#Bi-LSTM Implementation

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from sklearn.metrics import precision_recall_fscore_support

from sklearn.model_selection import StratifiedKFold

seed = 21
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
f_cvscores = []
prec_cvscores = []
recall_cvscores = []
iteration_count=1
for train_ind, test_ind in kfold.split(X, labels):
    print("Iteration count",iteration_count)
    iteration_count+=1
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False))
    model.add(Bidirectional(LSTM(46,dropout=0.22, recurrent_dropout=0.2)))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_labels = to_categorical(np.asarray(labels))
    model.fit(X[train_ind], model_labels[train_ind], epochs=23)
    y_pred=model.predict(X[test_ind])
    pred=[]
    for pair in y_pred:
        if pair[0]>pair[1]:
            pred.append(0)
        else:
            pred.append(1)
    test=[]
    for pair in model_labels[test_ind]:
        if pair[0]>pair[1]:
            test.append(0)
        else:
            test.append(1)
    f_cvscores.append(precision_recall_fscore_support(test,pred,average='weighted')[2])
    prec_cvscores.append(precision_recall_fscore_support(test,pred,average='weighted')[0])
    recall_cvscores.append(precision_recall_fscore_support(test,pred,average='weighted')[1])
    

('Iteration count', 1)
Epoch 1/23
1079/1079 [==============================] - 12s 12ms/step - loss: 0.6457 - acc: 0.6209
Epoch 2/23
1079/1079 [==============================] - 1s 1ms/step - loss: 0.5961 - acc: 0.6784
Epoch 3/23
1079/1079 [==============================] - 1s 1ms/step - loss: 0.5519 - acc: 0.7183
Epoch 4/23
1079/1079 [==============================] - 1s 1ms/step - loss: 0.5050 - acc: 0.7590
Epoch 5/23
1079/1079 [==============================] - 1s 1ms/step - loss: 0.4812 - acc: 0.7776
Epoch 6/23
1079/1079 [==============================] - 1s 1ms/step - loss: 0.4804 - acc: 0.7563
Epoch 7/23
1079/1079 [==============================] - 1s 1ms/step - loss: 0.4424 - acc: 0.7998
Epoch 8/23
1079/1079 [==============================] - 1s 1ms/step - loss: 0.4538 - acc: 0.7804
Epoch 9/23
1079/1079 [==============================] - 1s 1ms/step - loss: 0.4247 - acc: 0.8072
Epoch 10/23
1079/1079 [==============================] - 1s 1ms/step - loss: 0.4144 - acc: 0.8017
Epoc

1081/1081 [==============================] - 1s 1ms/step - loss: 0.5418 - acc: 0.7206
Epoch 4/23
1081/1081 [==============================] - 1s 1ms/step - loss: 0.5050 - acc: 0.7623
Epoch 5/23
1081/1081 [==============================] - 1s 1ms/step - loss: 0.4686 - acc: 0.7780
Epoch 6/23
1081/1081 [==============================] - 1s 1ms/step - loss: 0.4550 - acc: 0.7771
Epoch 7/23
1081/1081 [==============================] - 1s 1ms/step - loss: 0.4316 - acc: 0.7919
Epoch 8/23
1081/1081 [==============================] - 1s 1ms/step - loss: 0.4211 - acc: 0.8057
Epoch 9/23
1081/1081 [==============================] - 1s 1ms/step - loss: 0.4058 - acc: 0.8178
Epoch 10/23
1081/1081 [==============================] - 1s 1ms/step - loss: 0.3878 - acc: 0.8316
Epoch 11/23
1081/1081 [==============================] - 1s 1ms/step - loss: 0.3670 - acc: 0.8372A: 0s - loss: 0.3463 - acc: 
Epoch 12/23
1081/1081 [==============================] - 1s 1ms/step - loss: 0.3653 - acc: 0.8437
Epoch 13/2

In [110]:
np.mean(prec_cvscores)

0.7605219418073418

In [111]:
np.mean(recall_cvscores)

0.7557708868671436

In [112]:
np.mean(f_cvscores)

0.7553751974437712